In [103]:
# Imports you may need
from dataloader import *
from finance import *
import pandas as pd
import pageviewapi 

from finance import stock, compare
from quotebankexploration import *
from wikipedia import *

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [126]:
# Get some data frames
wiki_data = concat_wiki_files()
filtered_quotes = get_filtered_quotes()
speakers_id = pd.DataFrame(get_speakers_ids(filtered_quotes))
speakers_labels = get_speakers_labels()

In [140]:
year = 2015

test = speakers_labels.head(10)

test[str(year)] = test.label.apply(lambda label: get_page_views_per_year(label, year))

In [143]:
display(test)
test.to_pickle(path)

,speaker,qids,label,2015
0,steve jobs,[Q19837],Steve Jobs,6365607.0
1,ken whisenhunt,[Q1758635],Ken Whisenhunt,135492.0
2,robert macmillan,"[Q21453558, Q28094302]",Robert MacMillan,498.0
3,steve howard,"[Q23670647, Q2457386, Q7612886]",Steve Howard,9700.0
4,doug martin,"[Q18685889, Q27995830, Q3037945, Q3714598, Q53...",Doug Martin,3276.0
5,eddy cue,[Q224628],Eddy Cue,30460.0
6,tom harman,[Q7816093],Tom Harman,1829.0
7,none,[],None,NaN
8,trevor beattie,"[Q25190178, Q7839055]",Trevor Beattie,2640.0
9,michael phelps,[Q39562],Michael Phelps,713332.0


In [54]:
print(get_page_views_per_year('Tim Cook', 2015))

663329


In [ ]:
quotes_pageviews = filtered_quotes.set_index('speaker').join(speaker_pageviews.set_index('speaker'))

In [142]:
year = 2015

path = os.path.join('data/wiki_speaker_attributes/pageviews_'+str(year))

print(path)

data/wiki_speaker_attributes/pageviews_2015
